This is Munging and cleaning the 2019 data for comparison to the rest and seeing how well our model would perform on 2019 data

In [1]:
## import working modules
import pandas as pd
import numpy as np
import glob
import os
import unicodedata
from datetime import datetime
from functools import reduce


In [10]:
## loop through all the csvs, import them into a dictionary with edicted names
## first read all the csvs as raw files
## important note here, the output files save na values as "-", so those have to be dealt with. 
## you could do it later, but here I'm adding them in

df = [pd.read_csv(f,na_values = '-') for f in glob.glob('*.csv')]

## pull in the names and clean them
names = glob.glob('*.csv')
for i in range(len(names)):
    names[i]= names[i].replace("-","").replace("Natural Stat Trick","").replace(".csv","").replace(" ","")

## zip together for clean mapping and create dictionary
player_data = list(zip(names, df))
player_data_dict = dict(player_data)

In [11]:
## set index for joining
for key, value in player_data_dict.items():
    value.set_index(['Player','Year','Team'], inplace=True)
    
player_data_dict.keys()

dict_keys(['Combined_Player_Data', 'model_ready_data', 'PlayerSeasonTotals2014PKIndCounts', 'PlayerSeasonTotals2014PKOIRates', 'PlayerSeasonTotals2014PPindCounts', 'PlayerSeasonTotals2014PPOIRates', 'PlayerSeasonTotals2014SIndCounts', 'PlayerSeasonTotals2014SOIRates', 'PlayerSeasonTotals2015PKIndCounts', 'PlayerSeasonTotals2015PKOIRates', 'PlayerSeasonTotals2015PPIndCounts', 'PlayerSeasonTotals2015PPOIRates', 'PlayerSeasonTotals2015SIndCounts', 'PlayerSeasonTotals2015SOIRates', 'PlayerSeasonTotals2016PKindCounts', 'PlayerSeasonTotals2016PKOIRates', 'PlayerSeasonTotals2016PPindCounts', 'PlayerSeasonTotals2016PPOIRates', 'PlayerSeasonTotals2016SIndCounts', 'PlayerSeasonTotals2016SOIRates', 'PlayerSeasonTotals2017PKindCounts', 'PlayerSeasonTotals2017PKOIRates', 'PlayerSeasonTotals2017PPindCounts', 'PlayerSeasonTotals2017PPOIRates', 'PlayerSeasonTotals2017SIndCounts', 'PlayerSeasonTotals2017SOIRates', 'PlayerSeasonTotals2018PKindCounts', 'PlayerSeasonTotals2018PKOIRates', 'PlayerSeasonTota

In [12]:
## cleaning up column headers
## all those not recorded here I manually replaced " " with "_" and "%" with "Per" prior to saving
## if not for my meddling with the flat files you wouldn't have to do this, and arguably it's not 
## necissary here, but I prefer it to be more standardized
keys = player_data_dict.keys()
for i in keys:
    player_data_dict[i].columns = player_data_dict[i].columns.str.replace(r" ", "_")
    player_data_dict[i].columns = player_data_dict[i].columns.str.replace(r"%","Per")

player_data_2019 = pd.merge(player_data_dict['PlayerSeasonTotals2019SIndCounts'],player_data_dict['PlayerSeasonTotals2019PPindCounts'], left_index=True, right_index=True, suffixes=("",'_PPIndC'), how='outer').merge(player_data_dict['PlayerSeasonTotals2019PKindCounts'],left_index=True, right_index=True, suffixes=("",'_PkIndC'), how='outer').merge(player_data_dict['PlayerSeasonTotals2019PKOIRates'],left_index=True, right_index=True, suffixes=("",'_PkIndR'), how='outer').merge(player_data_dict['PlayerSeasonTotals2019PPOIRates'],left_index=True, right_index=True, suffixes=("",'_PPIndR'), how='outer').merge(player_data_dict['PlayerSeasonTotals2019SOIRates'],left_index=True, right_index=True, suffixes=("",'_SIndR'), how='outer').merge(player_data_dict['Playerbios2019b'],left_index=True, right_index=True, suffixes=("",'_Bios'), how='outer')


In [13]:
player_data_2019['Draft_Round'].fillna(100,inplace = True)
player_data_2019['Draft_Year'].fillna(1900,inplace = True)
player_data_2019['Round_Pick'].fillna(100,inplace = True)
player_data_2019['Overall_Draft_Position'].fillna(1000,inplace = True)
player_data_2019['Draft_Team'].fillna('Undrafted',inplace = True)
## this is the iteration to return each value
has_na = player_data_2019.isnull().sum() > 0
comb_player_data_cols = player_data_2019.columns[has_na]
comb_player_data_cols_2 = [col for col in player_data_2019.columns if col in comb_player_data_cols]
print(comb_player_data_cols_2)

['IPP', 'SHPer', 'Faceoffs_Per', 'Position_PPIndC', 'GP_PPIndC', 'TOI_PPIndC', 'Goals_PPIndC', 'Total_Assists_PPIndC', 'First_Assists_PPIndC', 'Second_Assists_PPIndC', 'Total_Points_PPIndC', 'IPP_PPIndC', 'Shots_PPIndC', 'SHPer_PPIndC', 'iCF_PPIndC', 'iFF_PPIndC', 'iSCF_PPIndC', 'iHDCF_PPIndC', 'Rush_Attempts_PPIndC', 'Rebounds_Created_PPIndC', 'PIM_PPIndC', 'Total_Penalties_PPIndC', 'Minor_PPIndC', 'Major_PPIndC', 'Misconduct_PPIndC', 'Penalties_Drawn_PPIndC', 'Giveaways_PPIndC', 'Takeaways_PPIndC', 'Hits_PPIndC', 'Hits_Taken_PPIndC', 'Shots_Blocked_PPIndC', 'Faceoffs_Won_PPIndC', 'Faceoffs_Lost_PPIndC', 'Faceoffs_Per_PPIndC', 'Position_PkIndC', 'GP_PkIndC', 'TOI_PkIndC', 'Goals_PkIndC', 'Total_Assists_PkIndC', 'First_Assists_PkIndC', 'Second_Assists_PkIndC', 'Total_Points_PkIndC', 'IPP_PkIndC', 'Shots_PkIndC', 'SHPer_PkIndC', 'iCF_PkIndC', 'iFF_PkIndC', 'iSCF_PkIndC', 'iHDCF_PkIndC', 'Rush_Attempts_PkIndC', 'Rebounds_Created_PkIndC', 'PIM_PkIndC', 'Total_Penalties_PkIndC', 'Minor_PkI

In [14]:
## these are the count values that I'm filling as zero
fill_zero = ['SHPer_PPIndC','IPP','SHPer','GP_PPIndC','TOI_PPIndC','Goals_PPIndC','Total_Assists_PPIndC','IPP_PPIndC','Shots_PPIndC','GP_PPIndC','TOI_PPIndC','IPP_PPIndC','IPP_PPIndC', 'Shots_PPIndC', 'SHPer_PPIndC', 'iSCF_PPIndC', 'iHDCF_PPIndC', 'Rush_Attempts_PPIndC', 'Rebounds_Created_PPIndC', 'PIM_PPIndC','Total_Penalties_PPIndC', 'Minor_PPIndC', 'Major_PPIndC', 'Misconduct_PPIndC', 'Penalties_Drawn_PPIndC', 'Giveaways_PPIndC', 'Takeaways_PPIndC', 'Hits_PPIndC', 'Hits_Taken_PPIndC', 'Shots_Blocked_PPIndC', 'Faceoffs_Won_PPIndC', 'Faceoffs_Lost_PPIndC','Faceoffs_Per_PPIndC','GP_PkIndC', 'TOI_PkIndC', 'Goals_PkIndC', 'Total_Assists_PkIndC', 'IPP_PkIndC', 'Shots_PkIndC', 'SHPer_PkIndC',    'iSCF_PkIndC', 'iHDCF_PkIndC', 'Rush_Attempts_PkIndC', 'Rebounds_Created_PkIndC', 'PIM_PkIndC', 'Total_Penalties_PkIndC', 'Minor_PkIndC', 'Major_PkIndC', 'Misconduct_PkIndC', 'Penalties_Drawn_PkIndC', 'Giveaways_PkIndC', 'Takeaways_PkIndC', 'Hits_PkIndC', 'Hits_Taken_PkIndC', 'Shots_Blocked_PkIndC', 'Faceoffs_Won_PkIndC', 'Faceoffs_Lost_PkIndC', 'PKTOI', 'PKTOI_GP', 'PKCF_60', 'PKCA_60', 'PKTOI', 'PKTOI_GP', 'PKCF_60', 'PKCA_60','PKFF_60', 'PKFA_60','PKSF_60', 'PKSA_60', 'PKGF_60', 'PKGA_60','PKSCF_60', 'PKSCA_60', 'PKHDCF_60', 'PKHDCA_60',  'PKHDGF_60', 'PKHDGA_60','PKLDCF_60', 'PKLDCA_60',  'PKMDCF_60', 'PKMDCA_60', 'PKMDGF_60', 'PKMDGA_60', 'PKLDGF_60', 'PKLDGA_60',  'PKPDO', 'PKOff.\xa0Zone_Faceoffs_60', 'PKNeu.\xa0Zone_Faceoffs_60', 'PKDef.\xa0Zone_Faceoffs_60', 'PKOnTFStarts_60',     'PKOffZoneFaceoffs_60', 'PKNeuZoneFaceoffs_60', 'PKDefZoneFaceoffs_60',   'PPTOI', 'PPTOI_GP', 'PPCF_60', 'PPCA_60', 'PPFF_60', 'PPFA_60',  'PPSF_60', 'PPSA_60',  'PPGF_60', 'PPGA_60', 'PPSCF_60', 'PPSCA_60','PPHDCF_60', 'PPHDCA_60',  'PPHDGF_60', 'PPHDGA_60',  'PPMDCF_60', 'PPMDCA_60','PPMDGF_60', 'PPMDGA_60',  'PPLDCF_60', 'PPLDCA_60', 'PPLDGF_60', 'PPLDGA_60', 'PPPDO', 'PPOff.\xa0Zone_Faceoffs_60', 'PPNeu.\xa0Zone_Faceoffs_60', 'PPDef.\xa0Zone_Faceoffs_60', 'PPOnTFStarts_60',  'PPOffZoneFaceoffs_60', 'PPNeuZoneFaceoffs_60', 'PPDefZoneFaceoffs_60',  'CFPer', 'FFPer', 'SFPer', 'GFPer', 'SCFPer', 'HDCFPer', 'HDGFPer', 'MDCFPer', 'MDGFPer', 'LDCFPer', 'LDGFPer', 'On-Ice_SHPer', 'On-Ice_SVPer', 'PDO', 'OffZoneStartPer', 'Off.\xa0Zone_Faceoff_Per']

for i in fill_zero:
    player_data_2019[i].fillna(0,inplace = True)
    
## mean values
fill_mean = ['PPLDGFPer', 'PPOn-Ice_SHPer', 'PPOn-Ice_SVPer',  'PPLDCFPer', 'PPMDGFPer','PPMDCFPer', 'PPHDGFPer','PPHDCFPer', 'PPSCFPer','PPGFPer','PPSFPer','PPFFPer','PPCFPer','PKOff.\xa0Zone_Faceoff_Per', 'PKOffZoneStartPer','PKLDGFPer', 'PKOn-Ice_SHPer', 'PKOn-Ice_SVPer','PPOffZoneStartPer','Faceoffs_Per','Faceoffs_Per_PPIndC','Faceoffs_Per_PPIndC','iCF_PPIndC',  'PKMDCFPer', 'PKMDGFPer',  'PKLDCFPer','iFF_PPIndC','iCF_PkIndC', 'iFF_PkIndC','Faceoffs_Per_PkIndC','PKCFPer','PKFFPer','PKSFPer','PKGFPer','PKSCFPer','PKHDCFPer', 'PKHDGFPer','PPOff.\xa0Zone_Faceoff_Per' ]

for i in fill_mean:
    player_data_2019[i].fillna(player_data_2019[i].mean(),inplace = True)

## the drop values
player_data_2019.drop(['Position_PPIndC', 'First_Assists_PPIndC', 'Second_Assists_PPIndC', 'Total_Points_PPIndC', 'Position_PkIndC', 'First_Assists_PkIndC', 'Second_Assists_PkIndC', 'Total_Points_PkIndC', 'Position_PkIndR', 'GP_PkIndR', 'Position_PPIndR', 'GP_PPIndR', 'Birth_State/Province','Position_PPIndR', 'GP_PPIndR','Position_PkIndR', 'GP_PkIndR',  'First_Assists_PkIndC', 'Second_Assists_PkIndC', 'Total_Points_PkIndC', 'Position_PkIndC','First_Assists','Second_Assists','Total_Points','Position_PPIndC','First_Assists_PPIndC', 'Second_Assists_PPIndC', 'Total_Points_PPIndC','Position_PPIndC','Birth_State/Province'], axis=1, inplace=True)
## dropping a few more excess columns I found, just to reduce junk variables
player_data_2019.drop(['GP_PPIndC','GP_PkIndC','Position_SIndR', 'GP_SIndR','Position_Bios'], axis=1, inplace=True)

## checking that all na are gone
has_na = player_data_2019.isnull().sum() > 0
player_data_2019_cols = player_data_2019.columns[has_na]
player_data_2019_cols_3 = [col for col in player_data_2019.columns if col in player_data_2019_cols]
print(player_data_2019_cols_3)

['Age', 'Date_of_Birth', 'Birth_City', 'Birth_Country', 'Nationality', 'Height_(in)', 'Weight_(lbs)']


In [15]:
player_data_2019.isnull().sum()

Position                  0
GP                        0
TOI                       0
Goals                     0
Total_Assists             0
IPP                       0
Shots                     0
SHPer                     0
iCF                       0
iFF                       0
iSCF                      0
iHDCF                     0
Rush_Attempts             0
Rebounds_Created          0
PIM                       0
Total_Penalties           0
Minor                     0
Major                     0
Misconduct                0
Penalties_Drawn           0
Giveaways                 0
Takeaways                 0
Hits                      0
Hits_Taken                0
Shots_Blocked             0
Faceoffs_Won              0
Faceoffs_Lost             0
Faceoffs_Per              0
TOI_PPIndC                0
Goals_PPIndC              0
                         ..
LDCF_60                   0
LDCA_60                   0
LDCFPer                   0
LDGF_60                   0
LDGA_60             

In [17]:
## since it's only 3 rows I'm just going to drop them here

player_data_2019.dropna(inplace=True)
## checking that all na are gone
has_na = player_data_2019.isnull().sum() > 0
player_data_2019_cols = player_data_2019.columns[has_na]
player_data_2019_cols_3 = [col for col in player_data_2019.columns if col in player_data_2019_cols]
print(player_data_2019_cols_3)

[]


In [21]:
## reset the index so we can use the year column
player_data_2019.reset_index(inplace=True)

## year imported as an int so need to convert that from 2018 into '1-1-2018'
## going to make a mini function here and then iterate row wise
def age_formula(x,y):
    return round((datetime.strptime(str(x), '%Y')-datetime.strptime(str(y), '%Y-%m-%d')).days/365)

player_data_2019['Age'] = player_data_2019.apply(lambda row: age_formula(row['Year'],row['Date_of_Birth']), axis=1)

## checking that it worked
player_data_2019['Age'].loc[player_data_2019['Player'] == 'Zdeno Chara']

ValueError: cannot insert level_0, already exists